In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6138/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_6138/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_12_7,0.967089,0.907697,0.889037,0.979180,0.013895,0.054795,0.049496,0.009704,0.044771,0.117878,0.956336,0.118761,74.552416,170.426484,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_19_7,0.967047,0.907594,0.901252,0.961854,0.013913,0.054856,0.044047,0.016593,0.044306,0.117952,0.956281,0.118836,74.549904,170.423971,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_19_3,0.967165,0.907411,0.900935,0.962821,0.013863,0.054965,0.044188,0.016173,0.044203,0.117742,0.956437,0.118624,74.557043,170.431111,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_12_3,0.967024,0.907349,0.888697,0.980007,0.013923,0.055001,0.049647,0.009318,0.044681,0.117995,0.956249,0.118878,74.548466,170.422534,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_19_0,0.967253,0.907266,0.900686,0.963571,0.013826,0.055051,0.044300,0.015847,0.044120,0.117583,0.956554,0.118464,74.562434,170.436501,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,model_4_39_8,0.933425,-5.160813,-27.339628,0.726717,0.028108,3.657325,12.641059,0.126104,0.050557,0.167655,0.758891,0.168911,201.143386,482.955040,"Hidden Size=[24], regularizer=0.02, learning_r..."
876,model_4_37_5,0.426812,-5.232525,-5.820609,-2.712931,0.242002,3.699896,3.042373,1.187654,0.063481,0.491937,-1.075869,0.495622,196.837616,478.649269,"Hidden Size=[24], regularizer=0.02, learning_r..."
877,model_4_40_8,0.932844,-5.335564,-26.746615,0.729050,0.028354,3.761065,12.376542,0.118049,0.046851,0.168385,0.756786,0.169646,201.126006,482.937660,"Hidden Size=[24], regularizer=0.02, learning_r..."
878,model_4_40_9,0.911250,-5.543552,-28.157923,0.617328,0.037471,3.884535,13.006064,0.166725,0.047791,0.193573,0.678581,0.195023,200.568395,482.380049,"Hidden Size=[24], regularizer=0.02, learning_r..."
